In [1]:
import random

suits = ['Hearts', 'Diamonds', 'Spades', 'Clubs']
ranks = ['Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace']
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10,
         'Queen':10, 'King':10, 'Ace':11}

playing = True

class Card:
    
    def __init__(self,suit,rank):
        self.suit = suit
        self.rank = rank
        self.value = values[rank]
    
    def __str__(self):
        return f"{self.rank} of {self.suit}"
      
class Deck:
    
    def __init__(self):
        self.deck = []  # start with an empty list
        for suit in suits:
            for rank in ranks:
                self.deck.append(Card(suit,rank))
    
    def __str__(self):
        return 'Deck has:\n\n'+ '\n'.join([card.__str__() for card in self.deck])

    def __len__(self):
        return len(self.deck)

    def shuffle(self):
        random.shuffle(self.deck)
        
    def deal(self):
        return self.deck.pop()
      
class Hand:
    def __init__(self):
        self.cards = []  # start with an empty list as we did in the Deck class
        self.value = 0   # start with zero value
        self.aces = 0    # add an attribute to keep track of aces
    
    def add_card(self,card):
        self.cards.append(card)
        self.value += card.value

        if card.rank == "Ace":
            self.aces+=1
    
    def adjust_for_ace(self):
        while self.value>21 and self.aces:
            self.aces-=1
            self.value-=10
            
class Chips:
    
    def __init__(self,total=100):
        self.total = total  # This can be set to a default value or supplied by a user input
        self.bet = 0
        
    def win_bet(self):
        self.total+= self.bet
    
    def lose_bet(self):
        self.total-= self.bet
        
def take_bet(chips):
    print(f"\n----------------------------------\n")
    while True:
        try:
            chips.bet = int(input("Please enter a bet you want to place: "))
        except:
            print("please enter a correct digit to place bet: ")
        else:
            if chips.bet>chips.total:
                print("you cannot place a bet greater than your chips: your avaialble chips are: {}".format(chips.total))
            else:
                print("bet for value: {} is placed \n--------------------------------".format(chips.bet))
                break
    
def hit(deck,hand):
    hand.add_card(deck.deal())
    hand.adjust_for_ace()
    
    
def hit_or_stand(deck,player):
    global playing  # to control an upcoming while loop
    while True:
        try:
            print(f"\n-------------------------------\npress 0 for Hit and press 1 for stand:")
            val = int(input("enter 0/1"))
        except:
            print("please enter a valis input")
        else:
            if val not in [0,1]:
                print(f"please enter either 0 for Hit or 1 for Stand")
            else:
                # correct input format and in range
                if val:
                    print("player stands, dealer's turn")
                    playing = False
                else:
                    hit(deck,player)
            break
    print(f"------------------------------------\n")
    
def show_some(player,dealer):
    print(f"\n--------------------------------")
    print(f"Dealer cards are: ")
    print(f"First card is hidden")
    print(f"{dealer.cards[1]}")
    print(f"dealer's hand value is unknown at this time: ")
    print(f"\n")
    print(f"Player cards are: ",*player.cards, sep="\n")
    print(f"value of player's hand is: ",player.value)
    print(f"\n--------------------------------")
    
    
    
def show_all(player,dealer):
    print(f"\n--------------------------------")
    print(f"Dealer cards are: ",*dealer.cards, sep="\n")
    print(f"value of dealer's hand is: ",dealer.value)
    print(f"\n")
    print(f"Player cards are: ",*player.cards, sep="\n")
    print(f"value of player's hand is: ",player.value)
    print(f"\n--------------------------------")
    
def player_busts(chips):
    print(f"Dealer Wins, Player Busts")
    chips.lose_bet()

def player_wins(chips):
    print(f"Player Wins")
    chips.win_bet()

def dealer_busts(chips):
    print(f"Player Wins, Dealer Busts")
    chips.win_bet()
    
def dealer_wins(chips):
    print(f"Dealer Wins")
    chips.lose_bet()
    
def push():
    print(f"Dealer and Player Tie. Push!!")
    
def play_again():
    while True:
        play_again  = input("enter y for playing again and n for quitting:")
        if play_again.lower()=='y':
            return True
        elif play_again.lower() == 'n':
            return False
        else:
            print(f"please enter a correct input")
            continue
          
print(f"Welcome to BlackJack Game!!")
player_chips = Chips()
while True:
    # check if chips are remaining for the user
    if player_chips.total<=0:
        print(f"\n Thanks for playing the game\nYou dont have enough chips to play the game, please restart the whole game to get fresh 100 chips\n")
        break
    
    # Create & shuffle the deck, deal two cards to each player
    game_deck = Deck()
    game_deck.shuffle()

    # create players
    player = Hand()
    dealer = Hand()
    for _ in range(2):
        player.add_card(game_deck.deal())
        dealer.add_card(game_deck.deal())
    # Set up the Player's chips
    print(f"-------------------------\nGame Started\n----------------------")
    
    # Prompt the Player for their bet
    take_bet(player_chips)
    
    # Show cards (but keep one dealer card hidden)
    show_some(player,dealer)   
    
    
    while playing:  # recall this variable from our hit_or_stand function
        
        # Prompt for Player to Hit or Stand
        hit_or_stand(game_deck,player)
        
        # Show cards (but keep one dealer card hidden)
        if playing:
            show_some(player,dealer)
        
        # If player's hand exceeds 21, run player_busts() and break out of loop
        if player.value > 21:
            player_busts(player_chips)
            break

    # If Player hasn't busted, play Dealer's hand until Dealer reaches 17
    else:
        # so player standed at a value and not busted then we will fall in this else block
        while dealer.value<17:
            hit(game_deck,dealer)
        # Show all cards
        show_all(player,dealer)
        # Run different winning scenarios
        # both delaer and player has now drawn the cards:

        if(dealer.value>21):
            dealer_busts(player_chips)
        elif dealer.value == player.value:
            push()
        elif dealer.value>player.value:
            dealer_wins(player_chips)
        else:
            player_wins(player_chips)
    
    # Inform Player of their chips total 
    print(f"player's remaining chips are: {player_chips.total}")
    playing = True
    
    # Ask to play again
    play_again_var = play_again()
    if not play_again_var:
        print(f"\nThanks For Playing BlackJack\n")
        break
    else:
        continue

Welcome to BlackJack Game!!
-------------------------
Game Started
----------------------

----------------------------------



Please enter a bet you want to place:  150


you cannot place a bet greater than your chips: your avaialble chips are: 100


Please enter a bet you want to place:  10


bet for value: 10 is placed 
--------------------------------

--------------------------------
Dealer cards are: 
First card is hidden
Eight of Clubs
dealer's hand value is unknown at this time: 


Player cards are: 
Nine of Clubs
Three of Hearts
value of player's hand is:  12

--------------------------------

-------------------------------
press 0 for Hit and press 1 for stand:


...

bet for value: 80 is placed 
--------------------------------

--------------------------------
Dealer cards are: 
First card is hidden
Seven of Clubs
dealer's hand value is unknown at this time: 


Player cards are: 
Five of Hearts
King of Diamonds
value of player's hand is:  15

--------------------------------

-------------------------------
press 0 for Hit and press 1 for stand:


enter 0/1 0


------------------------------------


--------------------------------
Dealer cards are: 
First card is hidden
Seven of Clubs
dealer's hand value is unknown at this time: 


Player cards are: 
Five of Hearts
King of Diamonds
Jack of Clubs
value of player's hand is:  25

--------------------------------
Dealer Wins, Player Busts
player's remaining chips are: 30


enter y for playing again and n for quitting: n



Thanks For Playing BlackJack

